# Kernel Matrix Approximation using random features (Mahalanobis norm)
This file does kernel matrix approximation for the Laplace kernel with M as a scale matrix. \
The kernel so calculated is $K_M(x,z) = e^{-\|x-z\|_M}$, where $\|x-z\|_M = \sqrt{(x-z)^TM(x-z)}$

In [1]:
import torch
from torchkernels.kernels.radial import laplacian
from torchkernels.feature_maps import LaplacianORF

In [2]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
n, d = 10, 10
U = torch.randn(d,d).to(DEVICE)
M = U@(U.T) 
p = int(3e4) # number of random features
X = torch.randn(n, d, device=DEVICE)

### Calculate Exact kernel matrix

In [3]:
kernel_matrix_exact = laplacian(X, X, length_scale=1., M=M)
kernel_matrix_exact_norm = torch.linalg.matrix_norm(kernel_matrix_exact, 'fro')

### Define random feature map

In [4]:
feature_map = LaplacianORF(input_dim=d, num_features=p, shape_matrix=M, device=DEVICE)

Calculate approximate kernel matrix

In [5]:
Phi = feature_map(X)
kernel_matrix_approx = Phi @ Phi.T

## Calculate approximation error

In [6]:
kernel_matrix_delta = kernel_matrix_exact - kernel_matrix_approx
kernel_matrix_delta_norm = torch.linalg.matrix_norm(kernel_matrix_delta, 'fro')
print(f"Approximation error in Frobenius norm: {kernel_matrix_delta_norm / kernel_matrix_exact_norm:.05f}")

Approximation error in Frobenius norm: 0.01575
